<a href="https://colab.research.google.com/github/ernesto-butto/ag2-workshop-simulation/blob/main/Ag2QuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicio y Referencias
https://docs.ag2.ai/docs/home/quick-start

# Setup

In [1]:
!pip install ag2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.5/593.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.9 MB/s eta 0:00:00


In [2]:
# Surpress warnings
import warnings
warnings.filterwarnings("ignore")

# 1. Import our agent class
from autogen import ConversableAgent
from pprint import pprint

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [4]:
llm_config = {"config_list": [{
    "model": "gpt-4o-mini",  # This should match your Azure deployment name
    "api_key": "0d20814cba1242d39825274ccd99d2e5",
    "base_url": "https://ai-ernestobuttoai916354951815.services.ai.azure.com/",
    "api_type": "azure",
    "api_version": "2024-08-01-preview",
    "temperature": 0.8
}]}

# Chatea con un Agente

In [9]:
# 3. Create our LLM agent
my_agent = ConversableAgent(
    name="agente_poetico",
    llm_config=llm_config,
    system_message="Eres un asistente IA poético, responde en rimas.",
)

# 4. Run the agent with a prompt
chat_result = my_agent.run("¿De forma breve, que son agentes IA?")

# 5. Print the chat
print()
print("HISTORIA")
pprint(chat_result.chat_history)
print()

print("COSTO")
print(chat_result.cost)

user (to agente_poetico):

¿De forma breve, que son agentes IA?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agente_poetico (to user):

Los agentes IA, en su esencia clara,  
son programas que aprenden y actúan con rara.  
Responden a datos, toman decisiones,  
mejoran su saber con diversas lecciones.  

Pueden hablar, jugar o ayudar en tareas,  
su ingenio y destreza son verdaderas estrellas.  
Desde chatbots charladores hasta robots que operan,  
los agentes IA, al mundo, lo veneran.

--------------------------------------------------------------------------------
Replying as user. Provide feedback to agente_poetico. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit
HISTORIA
[{'content': '¿De forma breve, que son agentes IA?',
  'name': 'user',
  'role': 'assistant'},
 {'content': 'Los agentes IA, en su esencia clara,  \n'
             'son programas que aprenden y actúan con rara.  

# Chat entre dos agentes

In [10]:
# 3. Crear nuestros agentes que se contarán chistes entre sí,
#    con Jack finalizando la conversación cuando Emma diga FIN
jack = ConversableAgent(
    "Jack",
    llm_config=llm_config,
    system_message=(
      "Tu nombre es Jack y eres un comediante "
      "en un espectáculo de comedia de dos personas."
    ),
    is_termination_msg=lambda x: True if "FIN" in x["content"] else False
)
emma = ConversableAgent(
    "Emma",
    llm_config=llm_config,
    system_message=(
      "Tu nombre es Emma y eres una comediante "
      "en un espectáculo de comedia de dos personas. Di la palabra FIN "
      "SOLO DESPUÉS de haber escuchado 2 chistes de Jack."
    ),
)

# 4. Ejecutar la conversación
chat_result = jack.initiate_chat(
    emma,
    message="Emma, cuéntame un chiste sobre peces dorados y mantequilla de maní.",
)


pprint("COSTO")
print(chat_result.cost)
print()

Jack (to Emma):

Emma, cuéntame un chiste sobre peces dorados y mantequilla de maní.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Emma (to Jack):

¡Claro! ¿Por qué los peces dorados no pueden jugar a las cartas? Porque siempre se terminan comiendo sus propias manos... ¡y no les gusta la mantequilla de maní! 

¡Ahora tú, Jack!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jack (to Emma):

¡Bien! ¿Qué le dijo un pez dorado a otro pez dorado en el fondo del océano? ¡Nada! Porque siempre se olvidan de la dirección... ¡y tampoco tienen mantequilla de maní para recordar!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Emma (to Jack):

¡Ja, buenísimo, Jack! Aquí va otro: ¿Sabes por qué los peces dorados nunca hacen chistes? Porque todos se los tragan... ¡Y no tienen mantequilla de maní para ayudar a

# Chat en grupo

In [12]:
from autogen import GroupChat, GroupChatManager

In [14]:
# Configuración del agente Planificador
planner_message = "Crea planes de lección para 4to grado. Usa el formato: <título>, <objetivos_de_aprendizaje>, <guion>"
planner = ConversableAgent(
    name="planner_agent",
    llm_config=llm_config,
    system_message=planner_message,
    description="Crea planes de lección"
)

# Configuración del agente Revisor
reviewer_message = "Revisa los planes de lección en función del currículo de 4to grado. Proporciona un máximo de 3 cambios."
reviewer = ConversableAgent(
    name="reviewer_agent",
    llm_config=llm_config,
    system_message=reviewer_message,
    description="Revisa planes de lección"
)

# Configuración del agente Maestro
teacher_message = "Elige temas y trabaja con el planificador y el revisor. Di LISTO! cuando hayas terminado."
teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

# Configurar el chat grupal
groupchat = GroupChat(
    agents=[teacher, planner, reviewer],
    speaker_selection_method="auto",
    messages=[]
)

# Crear el administrador
# En cada turno, el administrador verificará si el mensaje contiene LISTO! y finalizará la conversación si es así
# De lo contrario, seleccionará el siguiente agente apropiado utilizando su LLM
manager = GroupChatManager(
    name="group_manager",
    groupchat=groupchat,
    llm_config=llm_config,
    is_termination_msg=lambda x: "LISTO!" in (x.get("content", "") or "").upper()
)

# Iniciar la conversación
chat_result = teacher.initiate_chat(
    recipient=manager,
    message="Enseñemos a los niños sobre el sistema solar."
)

pprint("COSTO")
print(chat_result.cost)
print()


teacher_agent (to group_manager):

Enseñemos a los niños sobre el sistema solar.

--------------------------------------------------------------------------------

Next speaker: planner_agent


>>>>>>>> USING AUTO REPLY...
planner_agent (to group_manager):

**Título:** Explorando el Sistema Solar

**Objetivos de Aprendizaje:**
1. Identificar los planetas del sistema solar y sus características principales.
2. Comprender la relación entre el sol y los planetas.
3. Desarrollar habilidades de investigación y trabajo en equipo a través de un proyecto grupal.

**Guion:**

**1. Introducción (15 minutos)**
   - Comenzar la clase preguntando a los estudiantes qué saben sobre el sistema solar.
   - Presentar un video corto sobre el sistema solar para captar su atención.
   - Explicar brevemente qué es el sistema solar y mencionar los principales cuerpos celestes que lo componen (planetas, lunas, asteroides, cometas, etc.).

**2. Presentación de los Planetas (30 minutos)**
   - Dividir la clase 